# FINN flow  for MobileNet-V1
-------------------------------------------------------------

## Brevitas Export
--------------------------

<img src="img/brevitas_export.PNG" alt="Drawing" style="width: 600px;"/>

In [1]:
import finn.util.mobilenet_util as util

model = util.export_mobilenet()
model.save("mobilenet-v1.onnx")

## Visualization of MobileNet-v1 in Netron
-----------------------------------------------------
* Netron is a visualizer for neural networks and allows interactive investigation of network properties
* exported mobilenet consists of 16 layers:
    * 1 Convolutional Layer
    * 13 Depthwise-Separable Convolutional Layers 
        * 13 depthwise convolutions
        * 13 pointwise convolutions
    * 1 Pooling Layer
    * 1 Fully-connected Layer
    
-------------------------------------------------------------

In [1]:
from finn.util.visualization import showInNetron, showSrc

showInNetron("mobilenet-v1.onnx")

Serving 'mobilenet-v1.onnx' at http://0.0.0.0:5918


### Pytorch description of div node
------------------------------------------
<img src="img/div_pytorch.PNG" alt="Drawing" style="width: 300px;"/>

-----------------------------------------------------

## Network Preparation
-------------------------------------

<img src="img/NW_prep.PNG" alt="Drawing" style="width: 800px;"/>

-------------------------------------------------------------------------

## Streamlining
------------------------
* Main idea behind streamlining: eliminate floating point operations
    * moving them around 
    * collapsing them into one operation
    * transforming them into multithresholding nodes
  
--------------------------------------------

<img src="img/transformation.PNG" alt="Drawing" style="width: 700px;"/>

In [3]:
from finn.transformation.streamline import Streamline

showSrc(Streamline)

class Streamline(Transformation):
    """Apply the streamlining transform, see arXiv:1709.04060."""

    def apply(self, model):
        streamline_transformations = [
            ConvertSubToAdd(),
            ConvertDivToMul(),
            BatchNormToAffine(),
            ConvertSignToThres(),
            AbsorbSignBiasIntoMultiThreshold(),
            MoveAddPastMul(),
            MoveScalarAddPastMatMul(),
            MoveAddPastConv(),
            MoveScalarMulPastMatMul(),
            MoveScalarMulPastConv(),
            MoveAddPastMul(),
            CollapseRepeatedAdd(),
            CollapseRepeatedMul(),
            AbsorbAddIntoMultiThreshold(),
            FactorOutMulSignMagnitude(),
            AbsorbMulIntoMultiThreshold(),
            Absorb1BitMulIntoMatMul(),
            Absorb1BitMulIntoConv(),
            RoundAndClipThresholds(),
        ]
        for trn in streamline_transformations:
            model = model.transform(trn)
            model = model.transform(Remo

In [4]:
from finn.core.modelwrapper import ModelWrapper
import finn.util.mobilenet_util as util

model = ModelWrapper("mobilenet-v1.onnx")
# apply streamlining
model = model.transform(Streamline())
# apply mobilenet specific additional streamlining
model = util.additional_streamlining_mobilenet(model)
# lowering
model = util.lower_mobilenet(model)

model.save("mobilenet_streamlined.onnx")

In [2]:
showInNetron("mobilenet_streamlined.onnx")


Stopping http://0.0.0.0:5918
Serving 'mobilenet_streamlined.onnx' at http://0.0.0.0:5918


## Conversion to HLS Layers

<img src="img/conversion.PNG" alt="Drawing" style="width: 800px;"/>

In [6]:
model = ModelWrapper("mobilenet_lowered.onnx")
model = util.convert_to_hls_mobilenet(model)
model.save("mobilenet_hls.onnx")

### StreamingFCLayer function call
------------------------------------------
<img src="img/streamingfc.PNG" alt="Drawing" style="width: 700px;"/>

-----------------------------------------------------

In [3]:
showInNetron("mobilenet_hls.onnx")


Stopping http://0.0.0.0:5918
Serving 'mobilenet_hls.onnx' at http://0.0.0.0:5918


## Resource Allocation

<img src="img/folding.PNG" alt="Drawing" style="width: 900px;"/>

In [8]:
from finn.util.mobilenet_util import set_mobilenet_folding
model = ModelWrapper("mobilenet_hls.onnx")
# each tuple is (PE, SIMD, in_fifo_depth, ram_style) for a layer
folding = [
    (32, 3, 32, "auto"),
    (16, 16, 32, "auto"),
    (16, 16, 32, "auto"),
    (32, 16, 32, "auto"),
    (16, 16, 32, "auto"),
    (32, 16, 32, "auto"),
    (16, 16, 32, "auto"),
    (32, 16, 32, "auto"),
    (32, 16, 32, "auto"),
    (32, 16, 32, "auto"),
    (32, 16, 32, "auto"),
    (32, 16, 32, "auto"),
    (16, 16, 32, "auto"),
    (32, 16, 32, "auto"),
    (4, 4, 32, "auto"),
]
model = set_mobilenet_folding(model, folding)
model.save("mobilenet_folded.onnx")

In [4]:
showInNetron("mobilenet_folded.onnx")


Stopping http://0.0.0.0:5918
Serving 'mobilenet_folded.onnx' at http://0.0.0.0:5918


In [10]:
from finn.analysis.fpgadataflow.exp_cycles_per_layer import exp_cycles_per_layer
model = ModelWrapper("mobilenet_folded.onnx")
exp_cycles_dict = model.analysis(exp_cycles_per_layer)
util.show_exp_cycles_plot(model, exp_cycles_dict)

<img src="exp_cycles.png" alt="Drawing" style="width: 700px;"/>

# Vitis Build
----------------------
* IPI blocks are created for each layer
* Inserting data mover into graph 
    * to move data from DRAM into the accelerator and to move results back
* Stitching IPI blocks together
* Vitis build produces xclbin file and a driver

<img src="img/vitisbuild.PNG" alt="Drawing" style="width: 600px;"/>